In [1]:
import logging
from pathlib import Path

import numpy as np
import polars as pl

In [2]:
cfg = pl.Config()
cfg.set_tbl_rows(2000)
cfg.set_tbl_width_chars(200)
cfg.set_fmt_str_lengths(200)

polars.config.Config

In [3]:
parquet_files = list(Path("./").rglob("*.parquet"))

In [4]:
for i, file in enumerate(parquet_files):
    print(f"Processing {i} {file}")
    if file != Path("grocerystoredataset.parquet"):
        df = pl.read_parquet(file, columns=["ClassId"])
    else:
        df = pl.read_parquet(file, columns=["Coarse Class Name (str)"])
        df = df.rename({"Coarse Class Name (str)": "ClassId"})
    if i == 0:
        final_df = df.select("ClassId")
    else:
        final_df = pl.concat([final_df, df.select(pl.col("ClassId"))])

Processing 0 foodclassification.parquet
Processing 1 fruits360.parquet
Processing 2 FruitsAndVegetablesImageRecognitionDataset.parquet
Processing 3 fruitsclassification.parquet
Processing 4 grocerystoredataset.parquet


In [11]:
final_df = final_df.with_columns(
    pl.col("ClassId").str.to_lowercase().alias("ClassId")
)
unique_ingredients = final_df.unique().sort("ClassId")
unique_ingredients
unique_ing_list = unique_ingredients.write_parquet("unique_ingredients.parquet", compression="zstd", compression_level=3, statistics=True) 